In [69]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import re

df_input = pd.read_csv('../input_data_1~273.csv')
df_input=df_input.fillna('')
df_input.iloc[102]

pl_id                                                          103
pl_maker                                                   samsung
pl_group                                                   samsung
pl_model_code                                             SHW-M585
pl_name                                                      갤럭시코어
pl_model_name                                           GalaxyCore
Unnamed: 6                                                        
링크제목                                                        갤럭시 코어
나무위키링크단품         https://namu.wiki/w/%EA%B0%A4%EB%9F%AD%EC%8B%9...
나무위키링크시리즈        https://namu.wiki/w/%EA%B0%A4%EB%9F%AD%EC%8B%9...
Unnamed: 10                                                       
Name: 102, dtype: object

In [79]:
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
#     "모델명": "",
#     "모델코드": "",
#     "제조회사": "",
#     "출시일": "",
#     "운영체제": "",
#     "판매방식": "",

#     "화면크기(센치)": "",
#     "화면크기(인치)":"",
#     "패널종류": "",
#     "화면해상도": "", 
#     "ppi": "", 
#     "최대주사율": "",
#     "화면비": "",
#     "화면면적": "",
#     "화면폭": "",
#     "화면높이": "",
#     "HDR규격": "",

#     "AP종류": "", 
#     "코어갯수": "",
#     "코어클럭": "", 
#     "그래픽코어": "",
#     "NPU & DSP": "",
#     "Sensor Hub": "",
#     "시스템램": "",
#     "내장메모리": "",
#     "저장장치": "",
#     "외장메모리": "",
    
#     "5G": "",
#     "4G": "",
#     "WI-FI주파수": "",
#     "블루투스": "",
#     "위성항법": "",
#     "유심타입": "",
#     "듀얼유심": "",
    
#     "카메라타입": "",
#     "후면카메라": "",
#     "전면카메라": "",
#     "동영상촬영": "",
#     "Flash": "",
#     "사진촬영 해상도": "",
#     "전면 해상도": "",
#     "전면 동영상 해상도": "",
#     "조리개 값": "",
#     "카메라 특징": "",
#     "손떨림보정": "",
    
#     "손떨림방지": "",
#     "카메라플래시": "",
#     "HDR촬영지원": "",
#     "오토HDR": "",
#     "광학줌": "",
#     "지오태그": "",
#     "레이저 오토포커스": "",
#     "오토 포커스": "",
#     "아웃 포커스": "",
#     "터치 포커스": "",
#     "파노라마": "",
#     "TOF센서": "",
#     "야간모드 촬영": "",
    
#     "이어폰단자": "",
#     "스피커": "",
#     "고음질재생": "",
#     "사운드 기술": "",
    
#     "지문인식": "",
#     "얼굴인식": "",
#     "홍채인식": "",
#     "음성잠금해제": "",
#     "AI/인공지능": "",
#     "전자결제": "",
#     "터치펜": "",
#     "방수/방진": "",
    
#     "기타": "",
#     "충전단자": "",
#     "배터리용량": "",
#     "배터리타입": "",
#     "배터리특징": "",
#     "배터리장착방식": "",
#     "충전지원": "",
#     "고속충전기술": "",
#     "무선충전": "",
    
#     "색상": "",
    
#     "가로": "",
#     "세로": "",
#     "두께": "",
#     "무게": ""
    
 }
crawl_data = pd.DataFrame(my_dict,index=[0])
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################


#####################################################################################
######################### 함수 정의 ######################################
#####################################################################################
def hasxpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False
    
#####################################################################################
######################### 함수 정의 ######################################
#####################################################################################
    
    


# 초기화 --------------------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")
driver  = webdriver.Chrome('../chromedriver')
driver.implicitly_wait(3)
for index in range(40,len(df_input)):
#     try:
       
        temp_dict = dict()
        
        temp_dict["모델명"] = df_input.iloc[index][4]
        temp_dict["모델코드"] = df_input.iloc[index][3]
        temp_dict["제조회사"] = df_input.iloc[index][1]
        
        print(index,df_input.iloc[index]['나무위키링크단품'])
        if df_input.iloc[index]['나무위키링크단품'] != "":
            driver.get(df_input.iloc[index]['나무위키링크단품'])


            req = driver.page_source
            soup=BeautifulSoup(req, 'html.parser')
            table = ""
            if soup.select(".wiki-heading-content > .wiki-paragraph > div > .wiki-table-wrap > .wiki-table"):
                table = soup.select(".wiki-heading-content > .wiki-paragraph > div > .wiki-table-wrap > .wiki-table")[0]
            elif soup.select(".wiki-heading-content > .wiki-table-wrap > .wiki-table"):
                table = soup.select(".wiki-heading-content > .wiki-table-wrap > .wiki-table")[0]
            else:
                pass
                
            trs = table.select("tr")
            next_is_core_clock = 0
            for tr in trs:

                td = tr.select("td")

                #####################################프로세서파트##################################################################
                #####################################프로세서파트##################################################################
                if td[0].text == "프로세서" and "AP종류" not in temp_dict:
                    processor = td[1].text
                    temp_dict["프로세서"] = processor
                    
                    if processor.find("구성 내용 확인") != -1:
                        temp_dict["AP종류"] = re.search('.*\[ 구성 내용 확인 \]', processor).group().replace("[ 구성 내용 확인 ]","")
                    
                        proc_table = td[1].select(".wiki-table-wrap > table")
                        proc_trs = proc_table[0].select("tr")


                        for proc_tr in proc_trs:
                            proc_td = proc_tr.select("td")

                            if proc_td[0].text == "CPU":
                                next_is_core_clock = 1

                            elif next_is_core_clock == 1:
                                temp_dict["코어클럭"] = proc_td[0].text
                                next_is_core_clock = -1

                            elif proc_td[0].text == "GPU":
                                temp_dict["그래픽코어"] = proc_td[1].text

                            elif proc_td[0].text == "NPU & DSP":
                                temp_dict["NPU & DSP"] = proc_td[1].text

                            elif proc_td[0].text == "Sensor Hub":
                                temp_dict["Sensor Hub"] = proc_td[1].text

                            elif proc_td[0].text == "통신 모뎀":
                                temp_dict["통신 모뎀"] = proc_td[1].text
                    else:
                        #[구성내용 확인] 버튼이 없는경우 따로구현할것 
                        pass

                #####################################프로세서파트 끝##################################################################
                #####################################프로세서파트 끝##################################################################


                #####################################메모리 파트##################################################################
                #####################################메모리 파트##################################################################        
                elif td[0].text == "메모리":
                    memory = td[1].text
                    temp_dict["메모리"] = memory
                    
                    memorry_arr = td[1].text.split(',')

                    if len(memorry_arr) > 0:
                        temp_dict["시스템램"] = memorry_arr[0]
                    if len(memorry_arr) > 1:
                        in_mem = re.search('.*GB',memorry_arr[1]).group()
                        mem_dev = memorry_arr[1].replace(in_mem,"")
                        temp_dict["내장메모리"] = in_mem.replace(" GB","")
                        temp_dict["저장장치"] = mem_dev.replace(" 규격 내장 메모리","")
                    if len(memorry_arr) > 3:
                        temp_dict["외장메모리"] = memorry_arr[2]

                #####################################메모리 파트 끝##################################################################
                #####################################메모리 파트 끝##################################################################

                elif td[0].text == "디스플레이":
                    display = td[1].text
                    temp_dict["디스플레이"] = display
                    if display.find("세부 정보 확인")!= -1:
                        display_part1 = re.search('.*\[ 세부 정보 확인 \]', display).group().replace("[ 세부 정보 확인 ]","")
                        display_part1_list = display_part1.split(' ')
                        temp_dict["화면크기(인치)"] = display_part1_list[0]
                        temp_dict["화면크기(인치)"] = re.search('([0-9.]+인치)', display_part1).group() 



                        temp_dict["화면비"] = re.search('([0-9.]+:[0-9.] 비율)', display_part1).group().replace(" 비율","")
                        temp_dict["화면해상도"] = re.search('([0-9]+ x [0-9]+)', display_part1).group() 
                        temp_dict["ppi"] = re.sub('[\(\)]','',re.search('\([0-9-]+ ppi\)|\([0-9]+ppi\)',"".join(display_part1)).group())

                        disp_table = td[1].select(".wiki-table-wrap > table")
                        disp_trs = disp_table[0].select("tr")


                        for disp_tr in disp_trs:
                            disp_td = disp_tr.select("td")

                            if disp_td[0].text == "패널정보":
                                temp_dict["패널정보"] = disp_td[1].text

                            elif disp_td[0].text == "부가정보":
                                plus_info = disp_td[1].text
                                temp_dict["최대주사율"] = re.search('([0-9]+ Hz)|([0-9]+Hz)', plus_info).group()
                                
                    else:
                        #세부정보확인 버튼이 없는경우 따로구현할것
                        pass

                 #####################################디스플레이 파트 끝##################################################################
                #####################################디스플레이 파트 끝##################################################################

                elif td[0].text == "네트워크":
                    network = td[1].text
                    temp_dict["네트워크"] = network
                    if network.find("5G") != -1:
                        temp_dict["5G"] = "5G"
                    if network.find("4G") != -1:
                        temp_dict["4G"] = "4G"
                        
                #####################################네트워크 파트 끝##################################################################
                #####################################네트워크 파트 끝##################################################################
                  
                    
                    
                elif td[0].text == "근접통신":
                    proximity = td[1].text
                    temp_dict["근접통신"] = proximity
                    
                    if proximity.find("블루투스") != -1:
                        temp_dict["블루투스"] = re.sub('(블루투스,)',"",re.search('(블루투스 [0-9.]+)', proximity).group())
                    
                
                #####################################근접통신 파트 끝##################################################################
                #####################################근접통신 파트 끝##################################################################
                  
                
                
                elif td[0].text == "위성항법":
                    satelite = td[1].text
                    temp_dict["위성항법"] = satelite
                    
                    
                #####################################위성항법 파트 끝##################################################################
                #####################################위성항법 파트 끝##################################################################
                  
                
                elif td[0].text == "카메라":
                    camera = td[1].text
                    temp_dict["카메라"] = camera
                
                elif td[0].text == "배터리":
                    battery = td[1].text
                    temp_dict["배터리"] = battery
                
                elif td[0].text == "운영체제":
                    os = td[1].text
                    temp_dict["운영체제"] = os
                
                elif td[0].text == "규격":
                    size = td[1].text
                    temp_dict["규격"] = size
                
                elif td[0].text.find("색상") != -1 :
                    color = td[1].text
                    temp_dict["색상"] = color
                
                elif td[0].text == "단자정보":
                    terminal = td[1].text
                    temp_dict["단자정보"] = terminal
                
                elif td[0].text == "생체인식":
                    biometrics = td[1].text
                    temp_dict["생체인식"] = biometrics
                
                elif td[0].text == "기타":
                    etc = td[1].text
                    temp_dict["기타"] = etc


            crawl_data = crawl_data.append(pd.DataFrame(temp_dict,index=[0]))
#             print("=========================프로세서=============================")
#             if "AP종류" in temp_dict:
#                 print("[ AP종류 ]", temp_dict["AP종류"])
#             if "코어클럭" in temp_dict:
#                 print("[ 코어클럭 ]", temp_dict["코어클럭"])
#             if "그래픽코어" in temp_dict:
#                 print("[ 그래픽코어 ]", temp_dict["그래픽코어"])
#             if "NPU & DSP" in temp_dict:
#                 print("[ NPU & DSP ]", temp_dict["NPU & DSP"])
#             if "Sensor Hub" in temp_dict:
#                 print("[ Sensor Hub ]", temp_dict["Sensor Hub"])
#             if "통신 모뎀" in temp_dict:
#                 print("[ 통신 모뎀 ] ", temp_dict["통신 모뎀"])
                
#             print("===========================메모리===========================")
#             if "시스템램" in temp_dict:
#                 print("[ 시스템램 ]", temp_dict["시스템램"])
#             if "내장메모리" in temp_dict:
#                 print("[ 내장메모리 ]", temp_dict["내장메모리"])
#             if "저장장치" in temp_dict:
#                 print("[ 저장장치 ] ", temp_dict["저장장치"])
#             if "외장메모리" in temp_dict:
#                 print("[ 외장메모리 ]", temp_dict["외장메모리"])
                
#             print("===========================디스플레이===========================")
#             if "디스플레이" in temp_dict:
#                 print("[ 디스플레이 ]", temp_dict["디스플레이"])
#             if "화면크기(인치)" in temp_dict:
#                 print("[ 화면크기(인치) ]", temp_dict["화면크기(인치)"])
#             if "화면비" in temp_dict:
#                 print("[ 화면비 ]", temp_dict["화면비"])
#             if "화면해상도" in temp_dict:
#                 print("[ 화면해상도 ]", temp_dict["화면해상도"])
                
                
                
                
#             print("===========================네트워크===========================")
#             if "네트워크" in temp_dict:
#                 print("[ 네트워크 ]", temp_dict["네트워크"])
                
#             print("===========================근접통신===========================")
#             if "근접통신" in temp_dict:
#                 print("[ 근접통신 ]", temp_dict["근접통신"])
                
#             print("===========================카메라===========================")
#             if "카메라" in temp_dict:
#                 print("[ 카메라 ]", temp_dict["카메라"])
                
#             print("===========================운영체제===========================")
#             if "운영체제" in temp_dict:
#                 print("[ 운영체제 ]", temp_dict["운영체제"])
                
#             print("===========================규격===========================")
#             if "규격" in temp_dict:
#                 print("[ 규격 ]", temp_dict["규격"])
                
#             print("===========================색상===========================")
#             if "색상" in temp_dict:
#                 print("[ 색상 ]", temp_dict["색상"])
                
#             print("===========================단자정보===========================")
#             if "단자정보" in temp_dict:
#                 print("[ 단자정보 ]", temp_dict["단자정보"])
                
#             print("===========================생체인식===========================")
#             if "생체인식" in temp_dict:
#                 print("[ 생체인식 ]", temp_dict["생체인식"])
                
#             print("===========================기타===========================")
#             if "기타" in temp_dict:
#                 print("[ 기타 ]", temp_dict["기타"])

                
#     except Exception as e:    # 모든 예외의 에러 메시지를 출력할 때는 Exception을 사용
#         print('예외가 발생했습니다.', e)
#         empty_dict = {}
#         empty_dict["모델명"] = df_input.iloc[index][4]
#         empty_dict["모델코드"] = df_input.iloc[index][3]
#         empty_dict["제조회사"] = df_input.iloc[index][1]
#         crawl_data = crawl_data.append(pd.DataFrame(empty_dict,index=[0]))

        
        
        

                
                                             
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.
 
crawl_data



40 https://namu.wiki/w/SM-A700
41 https://namu.wiki/w/%EA%B0%A4%EB%9F%AD%EC%8B%9C%20A
42 https://namu.wiki/w/%EA%B0%A4%EB%9F%AD%EC%8B%9C%20J2%20Pro
43 https://namu.wiki/w/%EA%B0%A4%EB%9F%AD%EC%8B%9C%20J3(2017)


AttributeError: 'NoneType' object has no attribute 'group'

In [80]:
crawl_data

,모델명,모델코드,제조회사,프로세서,AP종류,코어클럭,NPU & DSP,Sensor Hub,통신 모뎀,메모리,시스템램,내장메모리,저장장치,외장메모리,디스플레이,화면크기(인치),화면비,화면해상도,ppi,패널정보,최대주사율,네트워크,4G,근접통신,블루투스,위성항법,카메라,배터리,운영체제,규격,색상,단자정보,생체인식,기타
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,갤럭시 A7,SM-A700,samsung,퀄컴 스냅드래곤 615 MSM8939 SoC. [ 구성 내용 확인 ]CPUARM ...,퀄컴 스냅드래곤 615 MSM8939 SoC.,GPU,퀄컴 Hexagon V50 DSP,DSP 대체,Qualcomm Gobi MDM9x25,"2 GB LPDDR3 SDRAM, 16 GB eMMC 5.0 규격 내장 메모리, m...",2 GB LPDDR3 SDRAM,16,eMMC 5.0,micro SDXC (규격상 2 TB,5.5인치 16:9 비율 FHD(1920 x 1080) Full-HD Super A...,5.5인치,16:9,1920 x 1080,-- ppi,Super AMOLED,60 Hz,[ 4G Network ]기본지원LTE Cat.4 FDDVoLTE 지원선택지원LT...,4G,"Wi-Fi 1/2/3/4, 블루투스 4.0+BLE, NFC[3], ANT+",블루투스 4.0,"GPS & A-GPS, GLONASS, Beidou","전면 500만 화소, 후면 1,300만 화소 AF 및 LED 플래시",내장형 Li-Ion 2600 mAh,안드로이드 4.4 (KitKat) → 5.0 (Lollipop) → 6.0 (Mar...,"76.2 x 151 x 6.3 mm, 141 g","미드나잇 블랙, 펄 화이트, 샴페인 골드","USB 2.0 micro Type-B x 1, 3.5 mm 단자 x 1일부 모델 하...",미지원,
0,갤럭시 A,SHW-M100,samsung,"TI OMAP 3440 SoC. ARM Cortex-A8 720 MHz CPU, P...",NaN,NaN,NaN,NaN,NaN,"384 MB LPDDR1 SDRAM, 1 GB OneNAND 내장 메모리[1], m...",384 MB LPDDR1 SDRAM,1,OneNAND 내장 메모리[1],NaN,3.7인치 WVGA(480 x 800) RG-BG 펜타일 서브픽셀 방식의 SMD A...,NaN,NaN,NaN,NaN,NaN,NaN,"HSDPA & HSUPA & UMTS, GSM & EDGE",NaN,"Wi-Fi 1/3/4, 블루투스 3.0",블루투스 3.0,NaN,"전면 30만 화소, 후면 500만 화소 AF 및 LED 플래시",Li-Ion 1500 mAh,안드로이드 2.1 (Eclair) → 2.2 (Froyo) → 2.3 (Ginger...,"59.8 x 119.5 x 12.5 mm, 128 g",NaN,NaN,NaN,지상파 DMB 지원
0,갤럭시 J2 Pro,SM-J250,samsung,삼성 엑시노스 7 Quad (7570) SoC. [ 구성 내용 확인 ]CPUARM...,삼성 엑시노스 7 Quad (7570) SoC.,GPU,미탑재,미탑재,(파트넘버 불명) 내장,"1 GB LPDDR3 SDRAM, 8 GB eMMC -.- 규격 내장 메모리, mi...",1 GB LPDDR3 SDRAM,8,eMMC -.-,micro SDXC (규격상 2 TB,5.0인치 16:9 비율 qHD(960 x 540) TFT-LCD (-- ppi) ...,5.0인치,16:9,960 x 540,-- ppi,TFT-LCD,60 Hz,[ 4G Network ]기본지원LTE Cat.4 FDD & TDD선택지원없음 [...,4G,"Wi-Fi 1/3/4, 블루투스 4.2",블루투스 4.2,"GPS & A-GPS, GLONASS, Beidou","전면 500만 화소, 후면 800만 화소 AF 및 LED 플래시",Li-Ion 2600 mAh,안드로이드 Go 8.1 (Oreo)Samsung Experience 8.1,"72.1 x 143.4 x 8.9 mm, 154 g","블랙, 골드, 블루","USB 2.0 micro Type-B x 1, 3.5 mm 단자 x 1듀얼심 지원",미지원,


In [67]:
crawl_data.to_csv('data_df_2.csv', # file path, file name
    ...: sep=',',   
    ...: na_rep='NaN',
                 encoding='utf-8-sig') 



In [ ]:
        
        
 
    
        
#         my_dict = {
#             #기본정보
#             "모델명": collet_one('/html/body/div[10]/div[3]/div[1]/div[2]/div[1]/div[1]/div/a/span',""),
#             "모델코드": df_input.iloc[index][3],
#             "제조회사": collet_one('//*[@id="product_specview"]/div[1]/form/div[1]/div[2]/span',""),
#             "출시일": collet_one('//*[@id="product_specview"]/div[1]/form/div[1]/div[4]/span',""),
#             "운영체제": collet_one('//*[@id="product_specview"]/div[1]/form/div[2]/div[4]/span',""),
#             "판매방식": "",
            
#             #화면정보
#             "화면크기(센치)": "",
#             "화면크기(인치)":collet_one('//*[@id="product_specview"]/div[1]/form/div[7]/div[2]/span',""),
#             "패널종류": collet_one('//*[@id="product_specview"]/div[1]/form/div[8]/div[4]/span',""),
#             "화면해상도": collet_one('//*[@id="product_specview"]/div[1]/form/div[7]/div[4]/span',""), 
#             "ppi": collet_one('//*[@id="product_specview"]/div[1]/form/div[8]/div[2]/span',""), 
#             "최대주사율": "",
#             "화면비": "",
#             "화면면적": "",
#             "화면폭": collet_one('//*[@id="product_specview"]/div[1]/form/div[9]/div[2]/span',""),
#             "화면높이": collet_one('//*[@id="product_specview"]/div[1]/form/div[9]/div[4]/span',""),
#             "HDR규격": "",
            
#             #시스템
#             "AP종류": collet_one('//*[@id="product_specview"]/div[1]/form/div[11]/div[2]/span',""), 
#             "코어갯수": collet_one('//*[@id="product_specview"]/div[1]/form/div[12]/div[2]/span',""),
#             "코어클럭": collet_one('//*[@id="product_specview"]/div[1]/form/div[12]/div[4]/span',""), 
#             "그래픽코어": collet_one('//*[@id="product_specview"]/div[1]/form/div[13]/div[2]/span',""),
#             "NPU & DSP": "",
#             "Sensor Hub": "",
#             "시스템램": collet_one('//*[@id="product_specview"]/div[1]/form/div[13]/div[4]/span',""),
#             "내장메모리": collet_one('//*[@id="product_specview"]/div[1]/form/div[14]/div[2]/span',""),
#             "저장장치": "",
#             "외장메모리": collet_one('//*[@id="product_specview"]/div[1]/form/div[14]/div[4]/span',""),
            
#             #네트워크
#             "5G": collet_one('//*[@id="product_specview"]/div[1]/form/div[25]/div[4]/span',"5G"),
#             "4G": collet_one('//*[@id="product_specview"]/div[1]/form/div[25]/div[4]/span',"4G"),
#             "WI-FI주파수": collet_one('//*[@id="product_specview"]/div[1]/form/div[26]/div[2]/span',""),
#             "블루투스": collet_one('//*[@id="product_specview"]/div[1]/form/div[26]/div[4]/span',""),
#             "위성항법": "",
#             "유심타입": collet_one('//*[@id="product_specview"]/div[1]/form/div[5]/div[4]/span',"유심타입"),
#             "듀얼유심": "",
            
#             #카메라
#             "카메라타입": "",
#             "후면카메라": "",
#             "전면카메라": "",
#             "동영상촬영": collet_one('//*[@id="product_specview"]/div[1]/form/div[18]/div[2]/span',"동영상촬영"),
#             "Flash": collet_one('//*[@id="product_specview"]/div[1]/form/div[17]/div[2]/span',""),
#             "사진촬영 해상도": collet_one('//*[@id="product_specview"]/div[1]/form/div[17]/div[4]/span',""),
#             "전면 해상도":  collet_one('//*[@id="product_specview"]/div[1]/form/div[19]/div[2]/span',""),
#             "전면 동영상 해상도": collet_one('//*[@id="product_specview"]/div[1]/form/div[19]/div[2]/span',"동영상해상도"),
#             "조리개 값": collet_one('//*[@id="product_specview"]/div[1]/form/div[16]/div[4]/span',""),
#             "카메라 특징": collet_one('//*[@id="product_specview"]/div[1]/form/div[20]/div[4]/span',""),
#             "손떨림보정": "",
            
#             #카메라기능
#             "손떨림방지": "",
#             "카메라플래시": "",
#             "HDR촬영지원": "",
#             "오토HDR": "",
#             "광학줌": "",
#             "지오태그": "",
#             "레이저 오토포커스": "",
#             "오토 포커스": "",
#             "아웃 포커스": "",
#             "터치 포커스": "",
#             "파노라마": "",
#             "TOF센서": "",
#             "야간모드 촬영": "",
            
#             #사운드
#             "이어폰단자": "",
#             "스피커": "",
#             "고음질재생": "",
#             "사운드 기술": "",
            
#             #보안/기능
#             "지문인식": collet_one('//*[@id="product_specview"]/div[1]/form/div[29]/div[2]/span',""),
#             "얼굴인식": collet_one('//*[@id="product_specview"]/div[1]/form/div[29]/div[2]/span',""),
#             "홍채인식": collet_one('//*[@id="product_specview"]/div[1]/form/div[29]/div[2]/span',""),
#             "음성잠금해제": "",
#             "AI/인공지능": "",
#             "전자결제": collet_one('//*[@id="product_specview"]/div[1]/form/div[29]/div[2]/span',""),
#             "터치펜": "",
#             "방수/방진": "",
            
#             #배터리
#             "기타": "",
#             "충전단자": "",
#             "배터리용량": collet_one('//*[@id="product_specview"]/div[1]/form/div[22]/div[2]/span',""),
#             "배터리타입": collet_one('//*[@id="product_specview"]/div[1]/form/div[22]/div[4]/span',""),
#             "배터리특징": collet_one('//*[@id="product_specview"]/div[1]/form/div[23]/div[2]/span',""),
#             "배터리장착방식": "",
#             "충전지원": "",
#             "고속충전기술": "",
#             "무선충전": collet_one('//*[@id="product_specview"]/div[1]/form/div[23]/div[4]/span',""),
            
#             #색상
#             "색상": "",
            
#             #규격
#             "가로": collet_one('//*[@id="product_specview"]/div[1]/form/div[4]/div[4]/span',"가로"),
#             "세로": collet_one('//*[@id="product_specview"]/div[1]/form/div[4]/div[4]/span',"세로"),
#             "두께": collet_one('//*[@id="product_specview"]/div[1]/form/div[4]/div[4]/span',"두께"),
#             "무게": driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[5]/div[2]/span').text
#         }
#         crawl_data = crawl_data.append(pd.DataFrame(my_dict,index=[0]))
#     else:
#         pass
    
    

,모델명,모델코드,제조회사,출시일,운영체제,판매방식,화면크기(센치),화면크기(인치),패널종류,화면해상도,ppi,최대주사율,화면비,화면면적,화면폭,화면높이,HDR규격,AP종류,코어갯수,코어클럭,그래픽코어,NPU & DSP,Sensor Hub,시스템램,내장메모리,저장장치,외장메모리,5G,4G,WI-FI주파수,블루투스,위성항법,유심타입,듀얼유심,카메라타입,후면카메라,전면카메라,동영상촬영,Flash,사진촬영 해상도,전면 해상도,전면 동영상 해상도,조리개 값,카메라 특징,손떨림보정,손떨림방지,카메라플래시,HDR촬영지원,오토HDR,광학줌,지오태그,레이저 오토포커스,오토 포커스,아웃 포커스,터치 포커스,파노라마,TOF센서,야간모드 촬영,이어폰단자,스피커,고음질재생,사운드 기술,지문인식,얼굴인식,홍채인식,음성잠금해제,AI/인공지능,전자결제,터치펜,방수/방진,기타,충전단자,배터리용량,배터리타입,배터리특징,배터리장착방식,충전지원,고속충전기술,무선충전,색상,가로,세로,두께,무게
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,아이폰11,A2221,애플,2019. 09,iOS 13,,,6.1,IPS,828 x 1792 픽셀,324 ppi,,,,64.99 mm,140.65 mm,,Apple A13 Bionic APL1W85,6개 (64 비트),2650 MHz,-,,,4 GB,64 GB / 128 GB / 256 GB,,미확인,LTE,LTE,미확인,5.0,,나노유심,,,,,3840 x 2160 (-)60 FPS,Dual LED,4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀)60 FPS,F/1.8,손떨림방지 카메라,,,,,,,,,,,,,,,,,,,,,,,,apple pay,,,,,3110 mAh,Li-Ion,급속충전\n일체형,,,,무선충전 지원,,75.7,150.9,8.3,194g


In [ ]:
#####이런식으로 변경?
row = [row.title, row.title, row.title]
series = pd.Seires(row, index=df.columns)
df = df.append(series, ignore_index = True)